In [2]:
import dask.dataframe as dd
import pathlib
import pandas as pd

In [3]:
from dask.distributed import Client

client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 15.52 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42013,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 15.52 GiB
Comm: tcp://127.0.0.1:38195,Total threads: 2
Dashboard: http://127.0.0.1:35351/status,Memory: 3.88 GiB
Nanny: tcp://127.0.0.1:46605,


In [ ]:
cases_dtype = {
    "ddl_case_id": "string[pyarrow]",
    "year": "int16",
    "state_code": "category",
    "dist_code": "category",
    "court_no": "category",
    "cino": "string[pyarrow]",
    'judge_position': "string[pyarrow]",
    "female_defendant": "category",
    "female_adv_def": "category",
    "female_adv_pet": "category",
    "type_name": "Int16",
    "purpose_name": "Int16",
    "disp_name": "Int16",
    "date_of_filing": "string[pyarrow]",
    "date_of_decision": "string[pyarrow]",
    "date_first_list": "string[pyarrow]",
    "date_last_list": "string[pyarrow]",
    "date_next_list": "string[pyarrow]",
}
date_parser = lambda x: pd.to_datetime(str(x), errors='coerce', format="%d-%m-%Y")
case_date_parser = lambda x: pd.to_datetime(str(x), errors='coerce', format="%Y-%m-%d")
futures = []

from datetime import date, timedelta


def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield pd.to_datetime(start_date + timedelta(n))


start_date = date(2010, 1, 1)
end_date = date(2019, 1, 1)
date_cases = pd.DataFrame({"date": daterange(start_date, end_date)})
date_cases.set_index("date")

for year in range(2010, 2019):
    finished_cases = dd.read_parquet(f"./Parquet/finished_cases/{year}/", engine="pyarrow", dtype=cases_dtype)
    courts = finished_cases[["state_code", "dist_code", "court_no"]].drop_duplicates()
    def func_maker(year_f, state_code, dist_code, court_no):
        def func():
            f_cases = dd.read_parquet(f"./Parquet/finished_cases/{year_f}/", engine="pyarrow", dtype=cases_dtype,
                                      filters=[("state_code", "==", state_code), ("dist_code", "==", dist_code),
                                               ("court_no", "==", court_no)])
            grouped_date_counts = f_cases.groupby(["date_of_filing", "date_of_decision"])[
                "state_code"].count().compute()
            date_cases_f = date_cases.copy()
            date_cases_f["cases"] = date_cases_f["date"].apply(
                lambda x: sum(
                    grouped_date_counts.loc[(grouped_date_counts.index.get_level_values('date_of_filing') <= x) & (
                            x <= grouped_date_counts.index.get_level_values('date_of_decision'))]))
            pathlib.Path(f"date_case_count/{year}/{state_code}/{dist_code}/").mkdir(parents=True, exist_ok=True)
            date_cases_f.to_csv(f"date_case_count/{year}/{state_code}/{dist_code}/date_case_count_{court_no}.csv")
            print(year_f, state_code, dist_code, court_no)

        return func


    def f_make(year_f):
        def f(x) -> None:
            futures.append(client.submit(func_maker(year_f, x["state_code"], x["dist_code"], x["court_no"])))
            return None

        return f


    courts.compute().apply(f_make(year), axis=1)
client.gather(futures)